### Overview

Just a quick run files for automl

In [7]:
PROJECT_ID='jchavezar-demo'
REGION='us-central1'
BUCKET_NAME='vtx-automl'
BUCKET_URI=f'gs://{BUCKET_NAME}'
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

In [6]:
import google.cloud.aiplatform as aiplatform
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## Create Dataset

In [8]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [9]:
dataset = aiplatform.ImageDataset.create(
    display_name="flowers_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

Creating ImageDataset
Create ImageDataset backing LRO: projects/569083142710/locations/us-central1/datasets/1144972035538026496/operations/4774818703214641152
ImageDataset created. Resource name: projects/569083142710/locations/us-central1/datasets/1144972035538026496
To use this ImageDataset in another session:
ds = aiplatform.ImageDataset('projects/569083142710/locations/us-central1/datasets/1144972035538026496')
Importing ImageDataset data: projects/569083142710/locations/us-central1/datasets/1144972035538026496
Import ImageDataset data backing LRO: projects/569083142710/locations/us-central1/datasets/1144972035538026496/operations/3214884382284185600
ImageDataset data imported. Resource name: projects/569083142710/locations/us-central1/datasets/1144972035538026496
projects/569083142710/locations/us-central1/datasets/1144972035538026496


### AutoML Image

In [10]:
automl = aiplatform.AutoMLImageTrainingJob(
    display_name="flowers_" + TIMESTAMP,
    prediction_type="classification",
    multi_label=False,
    model_type="MOBILE_TF_LOW_LATENCY_1",
    base_model=None,
)

print(automl)

In [11]:
model = automl.run(
    dataset=dataset,
    model_display_name="flowers_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/6399378950371409920?project=569083142710
AutoMLImageTrainingJob projects/569083142710/locations/us-central1/trainingPipelines/6399378950371409920 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLImageTrainingJob projects/569083142710/locations/us-central1/trainingPipelines/6399378950371409920 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLImageTrainingJob projects/569083142710/locations/us-central1/trainingPipelines/6399378950371409920 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLImageTrainingJob projects/569083142710/locations/us-central1/trainingPipelines/6399378950371409920 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLImageTrainingJob projects/569083142710/locations/us-central1/trainingPipelines/6399378950371409920 current state:
PipelineState.PIPELINE_STATE_RUNNING
AutoMLImageTrainingJob projects/569083142710/locations/us-central1/trainingPipeline

### Review Model Evaluation Scores

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

### Deploy the model

In [ ]:
endpoint = model.deploy()

In [ ]:
test_item = !gsutil cat $IMPORT_FILE | head -n1
if len(str(test_item[0]).split(",")) == 3:
    _, test_item, test_label = str(test_item[0]).split(",")
else:
    test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

import base64

from google.cloud import storage

# Copy the test image to the Cloud storage bucket as "test.jpg"
test_image_local = "{}/test.jpg".format(BUCKET_URI)
! gsutil cp $test_item $test_image_local

# Download the test image in bytes format
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(bucket_name=BUCKET_NAME)
test_content = bucket.get_blob("test.jpg").download_as_bytes()

# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{"content": base64.b64encode(test_content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

print(prediction)